In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from pykalman import KalmanFilter

# Load dataset
data = pd.read_csv('../datasets/CropSDEData/METEO_DEKADS_NUTS2_NL.csv')

# Feature Selection
target = 'PREC'

# Drop rows with missing values
data = data.dropna(subset=[target])

# Prepare data
y = data[target].values

# Ensure stationarity
def check_stationarity(y):
    from statsmodels.tsa.stattools import adfuller
    result = adfuller(y)
    return result[1]  # p-value of ADF test

if check_stationarity(y) > 0.05:
    print("Data is non-stationary. Applying differencing...")
    y = np.diff(y)

# Kalman Filter for Volatility Estimation
def estimate_volatility_kalman(y):
    kf = KalmanFilter(
        transition_matrices=[1],
        observation_matrices=[1],
        initial_state_mean=np.var(y),
        initial_state_covariance=1e-4,
        transition_covariance=1e-5,
        observation_covariance=1e-2
    )
    state_means, state_covariances = kf.filter(y**2)
    return np.sqrt(np.maximum(state_means.flatten(), 1e-5))  # Ensure non-negativity

# Estimate volatility from the observed data
volatility_estimates = estimate_volatility_kalman(y)

# Method of Moments (MoM) for Initial Guess
def heston_mom(y):
    dt = 1  # Assuming daily intervals
    dy = y[1:] - y[:-1]

    beta_mom = np.mean(y)
    alpha_mom = -np.mean(dy) / (beta_mom - np.mean(y[:-1]))
    theta_mom = np.var(dy)
    xi_mom = np.std(dy) / np.sqrt(dt)
    kappa_mom = 0.5  # Initial guess for mean-reversion speed

    return alpha_mom, beta_mom, kappa_mom, theta_mom, xi_mom

# Maximum Likelihood Estimation (MLE) for Heston Model
def heston_log_likelihood(params, data, vol_estimates):
    alpha, beta, kappa, theta, xi = params
    dt = 1  # Assuming daily intervals

    log_likelihood = 0
    for t in range(1, len(data)):
        V_t = vol_estimates[t]  # Kalman-filtered volatility
        residual = data[t] - (data[t-1] + alpha * (beta - data[t-1]) * dt)
        log_likelihood += -0.5 * np.log(2 * np.pi * V_t * dt) - (residual**2) / (2 * V_t * dt)

    return -log_likelihood  # Negative for minimization

# Bayesian Estimation Using MAP (Maximum A Posteriori)
def heston_log_posterior(params, data, vol_estimates):
    alpha, beta, kappa, theta, xi = params
    dt = 1  # Assuming daily intervals

    # Priors
    log_prior_alpha = norm.logpdf(alpha, loc=0.5, scale=0.5)
    log_prior_beta = norm.logpdf(beta, loc=np.mean(data), scale=np.std(data))
    log_prior_kappa = norm.logpdf(kappa, loc=0.5, scale=0.5)
    log_prior_theta = norm.logpdf(theta, loc=np.var(data), scale=0.5 * np.var(data))
    log_prior_xi = norm.logpdf(xi, loc=0.5, scale=0.5)

    log_prior = log_prior_alpha + log_prior_beta + log_prior_kappa + log_prior_theta + log_prior_xi

    # Likelihood
    log_likelihood = 0
    for t in range(1, len(data)):
        V_t = vol_estimates[t]
        residual = data[t] - (data[t-1] + alpha * (beta - data[t-1]) * dt)
        log_likelihood += -0.5 * np.log(2 * np.pi * V_t * dt) - (residual**2) / (2 * V_t * dt)

    return log_prior + log_likelihood  # Posterior = Prior + Likelihood

def heston_bayesian_map(data, vol_estimates, initial_guess):
    bounds = [(1e-5, None), (None, None), (1e-5, None), (1e-5, None), (1e-5, None)]
    res_map = minimize(
        lambda params: -heston_log_posterior(params, data, vol_estimates),  # Negative log-posterior for minimization
        initial_guess,
        method='L-BFGS-B',
        bounds=bounds
    )
    return res_map

# Run Method of Moments for Initial Estimates
alpha_mom, beta_mom, kappa_mom, theta_mom, xi_mom = heston_mom(y)
print("\nEstimated Heston Parameters using Method of Moments:")
print(f"Alpha: {alpha_mom}, Beta: {beta_mom}, Kappa: {kappa_mom}, Theta: {theta_mom}, Xi: {xi_mom}")

# MLE Estimation
initial_guess = [alpha_mom, beta_mom, kappa_mom, theta_mom, xi_mom]
bounds = [(1e-5, None), (None, None), (1e-5, None), (1e-5, None), (1e-5, None)]
res_mle = minimize(heston_log_likelihood, initial_guess, args=(y, volatility_estimates), method='L-BFGS-B', bounds=bounds)

if res_mle.success:
    alpha_mle, beta_mle, kappa_mle, theta_mle, xi_mle = res_mle.x
    print("\nEstimated Heston Parameters using Maximum Likelihood Estimation (MLE):")
    print(f"Alpha: {alpha_mle}, Beta: {beta_mle}, Kappa: {kappa_mle}, Theta: {theta_mle}, Xi: {xi_mle}")
else:
    print("\nMLE failed to converge.")
    print(f"Message: {res_mle.message}")

# Bayesian Estimation with MAP
print("\nRunning Bayesian Estimation Using MAP...")
res_map = heston_bayesian_map(y, volatility_estimates, initial_guess)
if res_map.success:
    alpha_map, beta_map, kappa_map, theta_map, xi_map = res_map.x
    print("\nEstimated Heston Parameters using Bayesian Estimation (MAP):")
    print(f"Alpha: {alpha_map}, Beta: {beta_map}, Kappa: {kappa_map}, Theta: {theta_map}, Xi: {xi_map}")
else:
    print("\nMAP Estimation failed to converge.")
    print(f"Message: {res_map.message}")


Estimated Heston Parameters using Method of Moments:
Alpha: -0.424106794233483, Beta: 1.9206313657407403, Kappa: 0.5, Theta: 4.973171265657719, Xi: 2.230060821066932

Estimated Heston Parameters using Maximum Likelihood Estimation (MLE):
Alpha: 0.967052845953362, Beta: 1.8822249344127342, Kappa: 0.5, Theta: 4.973171265657719, Xi: 2.230060821066932

Running Bayesian Estimation Using MAP...

Estimated Heston Parameters using Bayesian Estimation (MAP):
Alpha: 0.9669443496313676, Beta: 1.882228087479584, Kappa: 0.5, Theta: 2.5668653491720868, Xi: 0.4999242081724767
